In [24]:
import time

https://habr.com/ru/articles/800595/
https://habr.com/ru/articles/854664/
https://www.geeksforgeeks.org/nlp/byte-pair-encoding-bpe-in-nlp/
https://habr.com/ru/articles/517226/

# Токенизатор

## Первое знакомство

In [103]:
!pip install nltk
!pip install gensim
!pip install tiktoken
!pip install sentence_transformers


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


#### Разделение слов

In [20]:
from nltk.tokenize import RegexpTokenizer # основан на регулярках
from nltk.tokenize import TreebankWordTokenizer # Учитывает знаки препинания
from nltk.tokenize import WhitespaceTokenizer # Пробелы 
text = "This is an example text. Мой, кАрявый текст?"

In [32]:
tokenizers = {
    "reg": RegexpTokenizer(r'\w+'),
    "treebank": TreebankWordTokenizer(),
    "space": WhitespaceTokenizer()
}
for name, tokenizer in tokenizers.items():
    s_ = time.time()
    tokens = tokenizer.tokenize(text)
    t = time.time()-s_
    print(name+'-'*(80-len(name)))
    print(tokens, '\n', f'{t:.8f} сек')
    print('-'*80)

reg-----------------------------------------------------------------------------
['This', 'is', 'an', 'example', 'text', 'Мой', 'кАрявый', 'текст'] 
 0.00002003 сек
--------------------------------------------------------------------------------
treebank------------------------------------------------------------------------
['This', 'is', 'an', 'example', 'text.', 'Мой', ',', 'кАрявый', 'текст', '?'] 
 0.00007081 сек
--------------------------------------------------------------------------------
space---------------------------------------------------------------------------
['This', 'is', 'an', 'example', 'text.', 'Мой,', 'кАрявый', 'текст?'] 
 0.00001025 сек
--------------------------------------------------------------------------------


#### Векторизация слов

In [61]:
from gensim.models import Word2Vec
import numpy as np
sentences = [
    "Комната превратилась в арену для веселой суматохи.",
    "Пес с громким топотом носится за кошкой, которая грациозно отпрыгивает в сторону, сверкая белой шерсткой.",
    "Она делает вид, что убегает, затем резко разворачивается и бросается в погоню сама!",
    "Клубок меха и радости катается по полу.",
    "Вдруг оба замирают на полпути, уставившись на одинокую фигуру в дверях.",
    "Медленно, невероятно медленно, по ковру движется черепаха. Её панцирь поскрипывает.",
    "Кошка и собака смотрят с одинаковым недоумением: что это за странное, неторопливое существо?"
]

tokenizer = TreebankWordTokenizer()
data = [[w.lower() for w in tokenizer.tokenize(sen)] for sen in sentences]
    

# Обучение модели Word2Vec
model = Word2Vec(data, vector_size=10, window=3, min_count=1, workers=4)

# Получение векторного представления слова "Word2Vec"
vector_cat = model.wv["кошка"]
vector_dog = model.wv["пес"]
vector_tor = model.wv["черепаха."]
cos_dist = lambda a, b: round(1 - np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b)), 3)

print("кот и пес", cos_dist(vector_cat, vector_dog))
print("кот и черепаха", cos_dist(vector_cat, vector_tor))
print("черепаха и пес", cos_dist(vector_tor, vector_dog))

кот и пес 1.028
кот и черепаха 0.607
черепаха и пес 0.871


### Классификация по уровню
- слово: разбивает текст, ориентируясь на пробелы, знаки препинания, т.д.
- символ: разбивает текст на отдельные символы, иногда в качестве таких символов учитываются даже знаки препинания,
- подслово: текст разбивается на токены-подслова, которые иногда могут показаться просто тарабарщиной

#### Немного подробнее про подслова

In [129]:
from tiktoken._educational import *

# Train a BPE tokeniser on a small amount of text
enc = train_simple_encoding()

# Visualise how the GPT-4 encoder encodes text
enc = SimpleBytePairEncoding.from_tiktoken("cl100k_base")
enc.encode("hello world aaaaaaaaaaaa")

The current most common pair is b' ' + b' '
So we made b'  ' our 257th token
Now the first fifty words in our training data look like:
"""This is an educational implementation of the byte pair encoding algorithm."""

from __future__ import annotations

import collections

import regex

import tiktoken


class SimpleBytePairEncoding:
    def __init__(self, *, pat_str:


The current most common pair is b'  ' + b'  '
So we made b'    ' our 258th token
Now the first fifty words in our training data look like:
"""This is an educational implementation of the byte pair encoding algorithm."""

from __future__ import annotations

import collections

import regex

import tiktoken


class SimpleBytePairEncoding:
    def __init__(self, *, pat_str:


The current most common pair is b'  ' + b' '
So we made b'   ' our 259th token
Now the first fifty words in our training data look like:
"""This is an educational implementation of the byte pair encoding algorithm."""

from __future__ import annotation

[15339, 1917, 264, 70540, 33746]

In [141]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")
print(enc.n_vocab)
token_data = enc.encode("кошка пес и черепаха")
tokens = [enc.decode_single_token_bytes(number) for number in token_data]
for t in tokens:
    print(f"'{str(t, 'utf-8')}'")

200019
'ко'
'шка'
' пес'
' и'
' чер'
'еп'
'аха'


In [154]:
# Получение векторного представления слова "Word2Vec"
def get_token(text):
    token_data = enc.encode(text)
    tokens = [enc.decode_single_token_bytes(number) for number in token_data]
    return [str(t, 'utf-8') for t in tokens]


def get_emb(text, model):
    tokens = get_token(text)
    print(tokens)
    vecs = []
    for t in tokens:
        try:
            r = model.wv[t]
            vecs.append(r)
        except:
            pass
    print(vecs)
    return np.mean(vecs, axis=0)

sentences = [
"The sunbeam danced on the floor.",
"The cat Barsik decided to catch it.",
"He stalked it quietly and gracefully.",
"His tail twitched nervously with excitement.",
"The dog Sharik watched this from under the armchair.",
"Her patience was slowly running out.",
"Suddenly, she burst out of her hiding place with a loud bark.",
"Barsik jumped into the air from surprise.",
"He turned around in mid-air with feline agility.",
"Then he dashed away.",
"Sharik raced after him with a joyful yelp.",
"A merry chase began all through the house.",
"They ran around the sofa.",
"They raced under the dining table.",
"The curtains swayed from their frantic speed.",
"Barsik jumped onto the bookshelf.",
"He looked down at Sharik from above.",
"Mischievous sparks glittered victoriously in his eyes.",
"Sharik jumped up and down below, trying to reach him.",
"He barked cheerfully and excitedly.",
"Suddenly, Barsik jumped right onto the dog's back.",
"It was not an attack, but an invitation to play.",
"They tumbled across the carpet again in a fuzzy ball.",
"They made funny snorting sounds.",
"It seemed they had forgotten everything in the world.",
"There was not a hint of malice in their play.",
"They were just two friends frolicking with joy.",
"Suddenly, both froze at the same time.",
"Their attention was drawn by a movement at the door.",
"The turtle Tortilla was slowly crawling from there.",
"She leisurely moved her legs.",
"Her shell moved forward slowly.",
"She was the complete opposite of their chaotic game.",
"The cat and the dog looked at her with great surprise.",
"Barsik tilted his head to the side.",
"Sharik quieted down and stopped wagging his tail.",
"They didn't understand what this was.",
"They were amazed by her incredible slowness.",
"Tortilla completely ignored their astonished gazes.",
"She continued her important and unhurried journey."
]

data = [get_token(sen) for sen in sentences]

model = Word2Vec(data, vector_size=10, window=3, min_count=1, workers=4)


vector_cat = get_emb(" Barsik ", model)
vector_dog = get_emb(" Sharik ", model)
vector_tor = get_emb(" Tortilla ", model)
cos_dist = lambda a, b: round(1 - np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b)), 3)




print("кот и пес", cos_dist(vector_cat, vector_dog))
print("кот и черепаха", cos_dist(vector_cat, vector_tor))
print("черепаха и пес", cos_dist(vector_tor, vector_dog))

[' Bars', 'ik', ' ']
[array([-0.09004536, -0.00753587,  0.08192337,  0.07754123, -0.07174741,
       -0.03617473,  0.0316877 , -0.09554946,  0.01477656,  0.06486069],
      dtype=float32), array([ 0.0993943 , -0.0438615 , -0.00554422, -0.05667461,  0.03875517,
        0.0282416 ,  0.0695298 ,  0.06131249,  0.09531476,  0.09222362],
      dtype=float32)]
[' Sh', 'arik', ' ']
[array([-0.08077901, -0.07634935,  0.03007404, -0.02722349, -0.06919069,
       -0.08043492,  0.08384106,  0.0206832 , -0.0931285 , -0.04847604],
      dtype=float32), array([ 0.02325148, -0.04508699,  0.08426344, -0.09833159,  0.06774594,
        0.02949008, -0.04906881,  0.04415785, -0.01733209,  0.0669997 ],
      dtype=float32)]
[' Tort', 'illa', ' ']
[array([ 0.04416698,  0.06105732, -0.03118452, -0.04517281, -0.00438704,
        0.02587578, -0.03176076,  0.06137519,  0.04156606,  0.07746045],
      dtype=float32), array([-0.08347364,  0.09293731, -0.00033695, -0.01841249,  0.04614022,
       -0.03971864,  0.02

In [111]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("all-MiniLM-L6-v2")
# print(model.tokenizer.vocab_size)

# token_data = model.tokenize(["tokenizer tokenizes text into tokens"])
# tokens    =  model.tokenizer.convert_ids_to_tokens(tokenized_data["input_ids"][0])
# print(tokens)

In [162]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

examples  = ["EUR/USD", "EURUSD", "EUR-USD", "EUR.USD", "EUR:USD", "EUR vs USD", "EUR / USD", "Евро-Доллар", "Euro-Dollar", "EUR-USD Spot", "Евродоллар", "The Euro", "EURUSD=X", "EUR-USD Cash", "Евро к Доллару", "Euro to Dollar", "EUR-USD Fwd", "Fiber", "€/$", "EUR$"]

for exm in examples:
    token_data = enc.encode(exm)
    tokens = [enc.decode_single_token_bytes(number) for number in token_data]
    print('-'*40)
    print([f"'{str(t, 'utf-8')}'" for t in tokens])


----------------------------------------
["'EUR'", "'/USD'"]
----------------------------------------
["'EUR'", "'USD'"]
----------------------------------------
["'EUR'", "'-'", "'USD'"]
----------------------------------------
["'EUR'", "'.'", "'USD'"]
----------------------------------------
["'EUR'", "':'", "'USD'"]
----------------------------------------
["'EUR'", "' vs'", "' USD'"]
----------------------------------------
["'EUR'", "' /'", "' USD'"]
----------------------------------------
["'Е'", "'в'", "'ро'", "'-Д'", "'ол'", "'лар'"]
----------------------------------------
["'Euro'", "'-Dollar'"]
----------------------------------------
["'EUR'", "'-'", "'USD'", "' Spot'"]
----------------------------------------
["'Е'", "'в'", "'род'", "'ол'", "'лар'"]
----------------------------------------
["'The'", "' Euro'"]
----------------------------------------
["'EUR'", "'USD'", "'=X'"]
----------------------------------------
["'EUR'", "'-'", "'USD'", "' Cash'"]
-----------------